In [ ]:
import nnabla as nn

import nnabla.functions as F
import nnabla.parametric_functions as PF
import nnabla.solvers as S
from nnabla.monitor import tile_images

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
mb_size = 64
Z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
d_step = 3
lr = 1e-3
m = 5

In [ ]:
def G(z, hidden=[128, 128]):
    hs = []
    with nn.parameter_scope("G"):  # Parameter scope can be nested
        h = z
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("sigmoid_layer"):
            X = F.sigmoid(PF.affine(h, X_dim))
    return X

In [ ]:
def D(X, hidden=[128, 128]):
    hs = []
    with nn.parameter_scope("D"):  # Parameter scope can be nested
        h = X
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("reconstruction"):
            X_recon = PF.affine(h, X_dim)      
    return F.mean(F.sum((X - X_recon), axis=1))

In [ ]:
G_solver = S.Adam(lr)
with nn.parameter_scope("G"):
    G_solver.set_parameters(nn.get_parameters())
    
D_solver = S.Adam(lr)
with nn.parameter_scope("D"):
    D_solver.set_parameters(nn.get_parameters())

In [ ]:
def reset_grad():
    G_solver.zero_grad()
    D_solver.zero_grad()

In [ ]:
def show16(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    plt.show()

In [ ]:
for it in range(1000000):
    
    # Discriminator Update
    z = nn.Variable.from_numpy_array(np.random.randn(mb_size, Z_dim))
    X, _ = mnist.train.next_batch(mb_size)
    X = nn.Variable.from_numpy_array(X)

    G_sample = G(z)
    D_real = D(X)
    D_fake = D(G_sample)

    D_loss = D_real + F.relu(m - D_fake)
    
    D_loss.forward()
    D_loss.backward()
    D_solver.update()

    reset_grad()

    # Generator Update
    G_sample = G(z)
    D_fake = D(G_sample)

    G_loss = D_fake

    G_loss.forward()
    G_loss.backward()
    G_solver.update()

    reset_grad()
    
    # Generate and Show Samples 
    if it % 1000 == 0:
        idx = np.random.randint(0, 10)
        print('Step: {}, D_loss: {}, G_loss: {}'.format(it, D_loss.d, G_loss.d))
        
        samples = G(z).d[:16]
        show16(samples)